In [1]:
import os
import rootpath
import sys
sys.path.append(rootpath.detect())

import numpy as np
import pickle as pkl
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from scipy.stats import wilcoxon

In [3]:
REPORT_TABLE_DIR= '/home/finley/phd/papers/SAF-driven-EMO/tables/'

In [4]:
names = ['Mpoi', 'ParEgo', 'Saf_ei', 'Saf_$\\mu$', 'SmsEgo_ei', 'SmsEgo_$\\mu$', 'lhs']
true_names = ['MPoI', 'ParEGO', 'SAF: ei', 'SAF: $\mu$', 'SMS-EGO: ei', 'SMS-EGO: $\mu$', 'LHS']
D_names = {a:b for a, b in zip(names, true_names)}

In [5]:
def add_com(df, mask, com):
    best_inds =  [*zip(*np.where(mask == 1))] 
    for i, j in best_inds:
        df.iloc[i, j] = com + df.iloc[i, j]
    return df

In [6]:
def IQR(dist, axis):
    return np.percentile(dist, 75, axis=axis) - np.percentile(dist, 25, axis=axis)

In [7]:
def gen_table(d, result_min=0, result_max=6):
    h, w = d.shape
    headers = pd.MultiIndex.from_product([[c.replace('_', ' ').replace('wfg', 'WFG') for c in chosen_results[result_min:result_max]], ['Median', 'IQR']])
    df = pd.DataFrame(data=d.round(3).astype(str), index=true_names, columns=headers)
    return df, d

In [98]:
def gen_table_matrix(measure, result_min=0, result_max=6):
    if measure == 'igd+':
        invert=True
    else:
        invert=False
    # data matrix
    D = np.zeros((len(optimisers), len(chosen_results[result_min:result_max])*2))
    # mask of best values
    best_mask = np.zeros_like(D)
    # mask of equivalent values
    best_opts = []
    
    for i, prob in enumerate(chosen_results[result_min:result_max]):
        if invert:
            best_value = np.inf
        else:
            best_value = 0 
        best_index = 0
        for j, opt in enumerate(optimisers):
            med = np.median(results[prob]['data'][opt][measure], axis=0)[-1]
            D[j, i*2] = med 
            D[j, i*2+1] = IQR(results[prob]['data'][opt][measure], axis=0)[-1]
            if (med>best_value and invert == False) or (med<best_value and invert == True):
                best_value=med
                best_index = j
        best_mask[best_index, i*2] = True
        best_mask[best_index, i*2+1] = True
        best_opts.append(optimisers[best_index])
    
    eqv_mask = np.zeros_like(D)
    for i, prob in enumerate(chosen_results[result_min:result_max]):
        P = []
        for j, opt in enumerate(optimisers):
            if opt != best_opts[i]:
                hv = results[prob]['data'][opt][measure]
                bv = results[prob]['data'][best_opts[i]][measure]
                w, p = wilcoxon(bv[:, -1], hv[:, -1])
#                 print(p)
#                 print(bv[:, -1], "\n", hv[:, -1])
#                 print()
                P.append(p)
        p_inds = np.argsort(P)
        for ind, m in zip(p_inds, [r for r in range(len(P), 0, -1)]):
            if P[ind]>(0.05/m):
                if optimisers[ind] == best_opts[i]:
                    pass
                else:
                    eqv_mask[ind, i*2] = True
                    eqv_mask[ind, i*2+1] = True
    return D, best_mask, eqv_mask

In [99]:
sw = ((31*(31+1)*(2*31+1))/6)**0.5 
sw

102.05880657738459

In [100]:
parent_dir = os.path.join(rootpath.detect(), 'experiments/data')

In [101]:
results = {o:{'path':os.path.join(parent_dir, o, 'pkl_data/results_new.pkl')} for o in os.listdir(parent_dir) if o[:3] == 'wfg'}

In [102]:
for key, result in results.items():
    with open(result['path'], 'rb') as infile:
        results[key]['data'] = pkl.load(infile) 

In [103]:
chosen_results = [
    'wfg1_2obj_3dim', 
    'wfg1_3obj_4dim', 
    'wfg1_4obj_5dim', 
    'wfg2_2obj_6dim', 
    'wfg2_3obj_6dim', 
    'wfg2_4obj_10dim', 
    'wfg3_2obj_6dim', 
    'wfg3_3obj_10dim', 
    'wfg3_4obj_10dim_2', 
    'wfg4_2obj_6dim', 
    'wfg4_3obj_8dim', 
    'wfg4_4obj_8dim', 
    'wfg5_2obj_6dim', 
    'wfg5_3obj_8dim', 
    'wfg5_4obj_10dim', 
    'wfg6_2obj_10dim', 
    'wfg6_3obj_6dim', 
    'wfg6_4obj_12dim']

In [104]:
optimisers = list(results[chosen_results[0]]['data'].keys())
optimisers

['Mpoi', 'ParEgo', 'Saf_ei', 'Saf_$\\mu$', 'SmsEgo_ei', 'SmsEgo_$\\mu$', 'lhs']

In [105]:
d, bm, em = gen_table_matrix('hypervolume', result_min=0, result_max=6)
T_hv_1= gen_table(d, result_min=0, result_max=6)[0]
T_hv_1 = add_com(T_hv_1, bm, r'\best ')
T_hv_1 = add_com(T_hv_1, em, r'\statsimilar ')
with open(os.path.join(REPORT_TABLE_DIR, 'hv_table_1.tex'), "w") as outfile:
    print(T_hv_1.to_latex(index=True, escape=False), file=outfile)
T_hv_1

WFG1 2obj 3dim                     WFG1 3obj 4dim  \
                            Median                 IQR         Median   
MPoI                         8.119               1.494         67.189   
ParEGO                  \best 9.12         \best 0.381         83.195   
SAF: ei                      6.137               0.578         55.379   
SAF: $\mu$                    8.58                0.34          77.28   
SMS-EGO: ei     \statsimilar 8.735  \statsimilar 0.347         78.513   
SMS-EGO: $\mu$               8.898               0.502   \best 86.711   
LHS                          4.542               0.343         34.273   

                                  WFG1 4obj 5dim                      \
                        IQR               Median                 IQR   
MPoI                  9.635               20.003               2.914   
ParEGO               12.221               24.301               2.973   
SAF: ei               8.513               21.163               1.812   
SAF: $\mu$           11.424   \statsimilar 25.33  \statsimilar 5.039   
SMS-EGO: ei          10.102  \statsimilar 25.735  \statsimilar 2.996   
SMS-EGO: $\mu$  \best 3.705         \best 26.667         \best 1.544   
LHS                   2.858               12.809               0.903   

               WFG2 2obj 6dim                    WFG2 3obj 6dim  \
                       Median          IQR               Median   
MPoI                    3.822        0.641  \statsimilar 42.428   
ParEGO            \best 4.437  \best 0.338  \statsimilar 43.806   
SAF: ei                 3.859        0.369               41.745   
SAF: $\mu$              4.229        0.337  \statsimilar 42.894   
SMS-EGO: ei             3.624        0.763         \best 44.485   
SMS-EGO: $\mu$          4.109        0.454               42.212   
LHS                     4.094        0.228               39.339   

                                   WFG2 4obj 10dim               
                               IQR          Median          IQR  
MPoI             \statsimilar 5.34          47.566        5.897  
ParEGO          \statsimilar 4.201    \best 61.829  \best 7.354  
SAF: ei                      3.293          59.581       10.334  
SAF: $\mu$      \statsimilar 3.197          57.295        8.418  
SMS-EGO: ei            \best 5.492          54.464        9.021  
SMS-EGO: $\mu$               4.434          53.357       11.626  
LHS                          2.076           50.61        4.941

In [106]:
d, bm, em = gen_table_matrix('hypervolume', result_min=6, result_max=12)
T_hv_2= gen_table(d, result_min=6, result_max=12)[0]
T_hv_2 = add_com(T_hv_2, bm, r'\best ')
T_hv_2 = add_com(T_hv_2, em, r'\statsimilar ')
with open(os.path.join(REPORT_TABLE_DIR, 'hv_table_2.tex'), "w") as outfile:
    print(T_hv_2.to_latex(index=True, escape=False), file=outfile)
T_hv_2

WFG3 2obj 6dim              WFG3 3obj 10dim               \
                       Median          IQR          Median          IQR   
MPoI                    4.377         0.43           2.975        1.451   
ParEGO                  4.502        0.228            4.48        1.188   
SAF: ei                 5.123        0.137           5.128        0.385   
SAF: $\mu$              5.518         0.03           5.589        0.313   
SMS-EGO: ei             5.486        0.054           5.712        0.602   
SMS-EGO: $\mu$    \best 5.527  \best 0.021     \best 5.874  \best 0.408   
LHS                     3.732        0.141           2.702        0.279   

               WFG3 4obj 10dim 2                  WFG4 2obj 6dim  \
                          Median         IQR              Median   
MPoI                      17.671       8.095               1.844   
ParEGO                    27.913       2.203               2.456   
SAF: ei                   26.366       2.921  \statsimilar 2.489   
SAF: $\mu$                23.851       4.469         \best 2.633   
SMS-EGO: ei         \best 29.417  \best 3.19   \statsimilar 2.33   
SMS-EGO: $\mu$            29.357       2.425               2.554   
LHS                       13.436       3.282               2.093   

                                         WFG4 3obj 8dim                      \
                               IQR               Median                 IQR   
MPoI                         0.465               20.457               3.139   
ParEGO                       0.258               21.183               2.584   
SAF: ei          \statsimilar 0.13  \statsimilar 25.478  \statsimilar 1.542   
SAF: $\mu$             \best 0.213   \statsimilar 25.97  \statsimilar 1.397   
SMS-EGO: ei     \statsimilar 0.328               23.763               2.462   
SMS-EGO: $\mu$               0.204         \best 26.038          \best 2.04   
LHS                          0.113               18.023               1.015   

                      WFG4 4obj 8dim                      
                              Median                 IQR  
MPoI                         213.812               52.71  
ParEGO                       254.578              60.512  
SAF: ei                      291.237              21.915  
SAF: $\mu$                    292.89              15.001  
SMS-EGO: ei     \statsimilar 299.525  \statsimilar 30.93  
SMS-EGO: $\mu$         \best 306.483        \best 17.638  
LHS                          178.808                9.69

In [107]:
d, bm, em = gen_table_matrix('hypervolume', result_min=12, result_max=18)
T_hv_3= gen_table(d, result_min=6, result_max=12)[0]
T_hv_3 = add_com(T_hv_3, bm, r'\best ')
T_hv_3 = add_com(T_hv_3, em, r'\statsimilar ')
with open(os.path.join(REPORT_TABLE_DIR, 'hv_table_3.tex'), "w") as outfile:
    print(T_hv_3.to_latex(index=True, escape=False), file=outfile)
T_hv_3

WFG3 2obj 6dim              WFG3 3obj 10dim               \
                       Median          IQR          Median          IQR   
MPoI                     2.45        0.276          24.674        3.267   
ParEGO                  2.694        0.152           21.88        3.183   
SAF: ei                 2.251        0.234          20.909        2.281   
SAF: $\mu$              2.672         0.16    \best 26.007  \best 2.444   
SMS-EGO: ei       \best 2.823  \best 0.146          25.319        2.672   
SMS-EGO: $\mu$          2.732        0.198          23.986        2.543   
LHS                      1.56        0.081          15.803        0.733   

               WFG3 4obj 10dim 2               WFG4 2obj 6dim               \
                          Median           IQR         Median          IQR   
MPoI                     212.716        53.549          1.876        0.478   
ParEGO                   216.763        32.292          2.229        0.257   
SAF: ei                  205.022        27.537          2.571        0.201   
SAF: $\mu$          \best 259.04  \best 16.359    \best 2.951  \best 0.138   
SMS-EGO: ei              233.802        45.332           2.39        0.528   
SMS-EGO: $\mu$           214.564        33.576          2.343        0.454   
LHS                      153.719         5.534          1.137        0.115   

               WFG4 3obj 8dim              WFG4 4obj 8dim                
                       Median          IQR         Median           IQR  
MPoI                   27.247        3.467        197.858         37.45  
ParEGO                 21.039        2.726        187.395         23.81  
SAF: ei                29.516        1.898        198.523        18.302  
SAF: $\mu$             33.085        1.027        229.499        25.053  
SMS-EGO: ei      \best 33.836  \best 1.091  \best 261.581  \best 28.606  
SMS-EGO: $\mu$         33.192        1.341        251.868        23.481  
LHS                    16.276        1.171        134.995         4.638

In [108]:
d, bm, em = gen_table_matrix('igd+', result_min=0, result_max=6)
T_igd_1= gen_table(d, result_min=0, result_max=6)[0]
T_igd_1 = add_com(T_igd_1, bm, r'\best ')
T_igd_1 = add_com(T_igd_1, em, r'\statsimilar ')
with open(os.path.join(REPORT_TABLE_DIR, 'igd_table_1.tex'), "w") as outfile:
    print(T_igd_1.to_latex(index=True, escape=False), file=outfile)
T_igd_1

WFG1 2obj 3dim                          WFG1 3obj 4dim  \
                          Median                 IQR              Median   
MPoI                       1.058               0.181               1.476   
ParEGO               \best 0.966         \best 0.031  \statsimilar 1.301   
SAF: ei                    1.295               0.084               1.617   
SAF: $\mu$                 1.012               0.035                1.33   
SMS-EGO: ei     \statsimilar 1.0  \statsimilar 0.045               1.348   
SMS-EGO: $\mu$             0.979               0.054         \best 1.279   
LHS                        1.542               0.045               1.968   

                                        WFG1 4obj 5dim                      \
                               IQR              Median                 IQR   
MPoI                         0.187               1.863               0.162   
ParEGO          \statsimilar 0.124               1.643               0.139   
SAF: ei                      0.117               1.811               0.133   
SAF: $\mu$                   0.128   \statsimilar 1.62  \statsimilar 0.187   
SMS-EGO: ei                  0.109  \statsimilar 1.571  \statsimilar 0.113   
SMS-EGO: $\mu$         \best 0.042         \best 1.541         \best 0.066   
LHS                          0.053               2.279               0.053   

               WFG2 2obj 6dim                   WFG2 3obj 6dim  \
                       Median          IQR              Median   
MPoI                    0.462        0.165         \best 0.539   
ParEGO            \best 0.368  \best 0.062                0.61   
SAF: ei                 0.511        0.089               0.731   
SAF: $\mu$              0.427        0.074  \statsimilar 0.687   
SMS-EGO: ei             0.523        0.152  \statsimilar 0.544   
SMS-EGO: $\mu$          0.392        0.134               0.644   
LHS                     0.444        0.071               0.722   

                                   WFG2 4obj 10dim               
                               IQR          Median          IQR  
MPoI                   \best 0.106           0.604        0.127  
ParEGO                       0.185     \best 0.477  \best 0.162  
SAF: ei                      0.158           0.553        0.191  
SAF: $\mu$      \statsimilar 0.169           0.568        0.255  
SMS-EGO: ei     \statsimilar 0.253           0.586         0.14  
SMS-EGO: $\mu$               0.221           0.622        0.242  
LHS                          0.151           0.545        0.086

In [109]:
d, bm, em = gen_table_matrix('igd+', result_min=6, result_max=12)
T_igd_2= gen_table(d, result_min=6, result_max=12)[0]
T_igd_2 = add_com(T_igd_2, bm, r'\best ')
T_igd_2 = add_com(T_igd_2, em, r'\statsimilar ')

with open(os.path.join(REPORT_TABLE_DIR, 'igd_table_2.tex'), "w") as outfile:
    print(T_igd_2.to_latex(index=True, escape=False), file=outfile)
T_igd_2

WFG3 2obj 6dim                         WFG3 3obj 10dim  \
                            Median                 IQR              Median   
MPoI                         0.241               0.092               0.375   
ParEGO                       0.199               0.044               0.287   
SAF: ei                      0.094               0.026               0.193   
SAF: $\mu$      \statsimilar 0.026  \statsimilar 0.005               0.113   
SMS-EGO: ei                  0.032               0.011  \statsimilar 0.104   
SMS-EGO: $\mu$         \best 0.025         \best 0.003         \best 0.095   
LHS                          0.363               0.041               0.564   

                                   WFG3 4obj 10dim 2               \
                               IQR            Median          IQR   
MPoI                         0.192               0.3        0.197   
ParEGO                       0.176             0.291        0.068   
SAF: ei                       0.05             0.207        0.037   
SAF: $\mu$                   0.028             0.199        0.053   
SMS-EGO: ei     \statsimilar 0.053       \best 0.124  \best 0.038   
SMS-EGO: $\mu$         \best 0.041             0.132        0.034   
LHS                          0.063             0.588        0.046   

                    WFG4 2obj 6dim                          WFG4 3obj 8dim  \
                            Median                 IQR              Median   
MPoI                         0.271               0.085               0.499   
ParEGO                       0.157               0.043               0.461   
SAF: ei         \statsimilar 0.151  \statsimilar 0.023   \statsimilar 0.33   
SAF: $\mu$             \best 0.129         \best 0.037         \best 0.314   
SMS-EGO: ei     \statsimilar 0.192  \statsimilar 0.075  \statsimilar 0.379   
SMS-EGO: $\mu$               0.141               0.034               0.315   
LHS                          0.284               0.046               0.782   

                                        WFG4 4obj 8dim                     
                               IQR              Median                IQR  
MPoI                         0.117               0.822              0.379  
ParEGO                       0.139               0.718              0.326  
SAF: ei         \statsimilar 0.055                 0.5              0.107  
SAF: $\mu$             \best 0.042               0.497              0.064  
SMS-EGO: ei      \statsimilar 0.12  \statsimilar 0.474  \statsimilar 0.09  
SMS-EGO: $\mu$               0.059         \best 0.431        \best 0.061  
LHS                          0.105                1.58                0.3

In [110]:
d, bm, em = gen_table_matrix('igd+', result_min=12, result_max=18)
T_igd_3= gen_table(d, result_min=12, result_max=18)[0]
T_igd_3 = add_com(T_igd_3, bm, r'\best ')
T_igd_3 = add_com(T_igd_3, em, r'\statsimilar ')
with open(os.path.join(REPORT_TABLE_DIR, 'igd_table_3.tex'), "w") as outfile:
    print(T_igd_3.to_latex(index=True, escape=False), file=outfile)
T_igd_3

WFG5 2obj 6dim                          WFG5 3obj 8dim  \
                           Median                 IQR              Median   
MPoI                        0.205               0.089  \statsimilar 0.411   
ParEGO          \statsimilar 0.11  \statsimilar 0.026  \statsimilar 0.443   
SAF: ei                     0.244               0.069               0.598   
SAF: $\mu$                  0.131               0.028         \best 0.369   
SMS-EGO: ei           \best 0.096         \best 0.041                0.37   
SMS-EGO: $\mu$              0.116               0.047               0.474   
LHS                          0.39               0.031                0.69   

                                   WFG5 4obj 10dim               \
                               IQR          Median          IQR   
MPoI            \statsimilar 0.157           0.993        0.359   
ParEGO          \statsimilar 0.108            0.91         0.22   
SAF: ei                      0.118           1.091        0.329   
SAF: $\mu$             \best 0.117     \best 0.711  \best 0.125   
SMS-EGO: ei                  0.139           0.943        0.396   
SMS-EGO: $\mu$               0.199           1.251        0.586   
LHS                          0.045           1.173        0.107   

               WFG6 2obj 10dim                   WFG6 3obj 6dim  \
                        Median          IQR              Median   
MPoI                     0.297        0.094               0.292   
ParEGO                   0.202        0.052               0.472   
SAF: ei                  0.146        0.036                0.24   
SAF: $\mu$         \best 0.073  \best 0.029  \statsimilar 0.142   
SMS-EGO: ei              0.185        0.108         \best 0.132   
SMS-EGO: $\mu$           0.193        0.104               0.149   
LHS                      0.568        0.043               0.623   

                                   WFG6 4obj 12dim               
                               IQR          Median          IQR  
MPoI                         0.103            0.71        0.147  
ParEGO                       0.077           0.881         0.13  
SAF: ei                      0.055           0.813        0.052  
SAF: $\mu$      \statsimilar 0.027            0.65        0.092  
SMS-EGO: ei            \best 0.027     \best 0.573  \best 0.076  
SMS-EGO: $\mu$                0.03           0.615        0.072  
LHS                          0.039           1.235         0.07